In [83]:
## Dependencies

import pandas as pd
import os


In [84]:
# years = [2010:2023]

year = 2023

In [90]:
## laod test data

base_dir = 'TEMP/clean_tables/game_level/new/'

# rename files in the directory
# list all the files in the directory
files = os.listdir(base_dir)

print(files)


# drop first 10 characters
# add 'summary_year' to the front of the file name
# save to new directory

for file in files:

    # load file
    df = pd.read_csv(base_dir + file)

    # drop first 10 characters from file name
    file = file[11:]
    

    # add 'summary_year' to the front of the file name
    df.to_csv('TEMP/clean_tables/game_level/' + 'full_game_index' + file)

    print('done with ' + file)




['_NEW_SCRAPE_2010.csv', '_NEW_SCRAPE_2011.csv', '_NEW_SCRAPE_2012.csv', '_NEW_SCRAPE_2013.csv', '_NEW_SCRAPE_2014.csv', '_NEW_SCRAPE_2015.csv', '_NEW_SCRAPE_2016.csv', '_NEW_SCRAPE_2017.csv', '_NEW_SCRAPE_2018.csv', '_NEW_SCRAPE_2019.csv', '_NEW_SCRAPE_2020.csv', '_NEW_SCRAPE_2021.csv', '_NEW_SCRAPE_2022.csv']
done with _2010.csv
done with _2011.csv
done with _2012.csv
done with _2013.csv
done with _2014.csv
done with _2015.csv
done with _2016.csv
done with _2017.csv
done with _2018.csv
done with _2019.csv
done with _2020.csv
done with _2021.csv
done with _2022.csv


In [ ]:
## NEWWLY REFACTORED FUNCTION MON 1PM CHATGPT

import pandas as pd

def process_stats(df):
    ## Create a column with the result of each game win, loss, or tie
    def calc_result(row):
        if row['teamScore'] > row['opponentScore']:
            return 'W'
        elif row['teamScore'] < row['opponentScore']:
            return 'L'
        else:
            return 'T'
    df['result'] = df.apply(calc_result, axis=1)

    def process_slice(df_slice, prefix):
        grouped = df_slice.groupby('teamName')

        # Save value counts to a new dataframe with the team name as the index and columns for each result type
        results_frame = pd.DataFrame(grouped['result'].value_counts()).rename(columns={'result':'count'}).reset_index()

        # Reshape results into separate columns for each result type using pivot table
        results_frame = results_frame.pivot(index='teamName', columns='result', values='count')

        # Replace NaN values with 0
        results_frame = results_frame.fillna(0)

        # Rename the columns
        results_frame.columns = [f'{prefix}_{col}' for col in results_frame.columns]

        # Rest of the function
        ...



        stat = grouped.agg({
            'teamScore': ['count', 'mean', 'median', 'sum'],
            'opponentScore': ['count', 'mean', 'median', 'sum']
        }).reset_index()

        # simplify the column names
        stat.columns = ['teamName', 
                        
                        f'{prefix}_games_played',
                        f'{prefix}_runs_scored_mean',
                        f'{prefix}_runs_scored_median',
                        f'{prefix}_runs_scored_total',

                        f'{prefix}_runs_allowed_count',
                        f'{prefix}_runs_allowed_mean',
                        f'{prefix}_runs_allowed_median',
                        f'{prefix}_runs_allowed_total']

        # Add the result count columns to the stat dataframe
        stat = stat.merge(results_frame, how='left', on='teamName')

        # Calculate winning percentage and run differential (total and mean)
        stat[f'{prefix}_win_pct'] = stat[f'{prefix}_W'] / stat[f'{prefix}_games_played']
        stat[f'{prefix}_run_diff_total'] = stat[f'{prefix}_runs_scored_total'] - stat[f'{prefix}_runs_allowed_total']
        stat[f'{prefix}_run_diff_mean'] = stat[f'{prefix}_runs_scored_mean'] - stat[f'{prefix}_runs_allowed_mean']

        return stat


    # Process data slices
    slices = {
        'playoff': df[df['postSeasonInfo'].notnull()],
        'regular_season': df[df['postSeasonInfo'].isnull()],
        'home': df[(df['postSeasonInfo'].isnull()) & (df['homeOrAway'] == 'H')],
        'away': df[(df['postSeasonInfo'].isnull()) & (df['homeOrAway'] == 'A')],
    }
    combined_stat = None
    for prefix, df_slice in slices.items():
        slice_stat = process_slice(df_slice, prefix)
        if combined_stat is None:
            combined_stat = slice_stat
        else:
            combined_stat = combined_stat.merge(slice_stat, how='left', on='teamName')

    return combined_stat


In [ ]:
## Run the function on the test data and display the results

test_df = process_stats(df)

## Display the results
test_df.head()






,teamName,playoff_games_played,playoff_runs_scored_mean,playoff_runs_scored_median,playoff_runs_scored_total,playoff_runs_allowed_count,playoff_runs_allowed_mean,playoff_runs_allowed_median,playoff_runs_allowed_total,playoff_L,...,away_runs_allowed_count,away_runs_allowed_mean,away_runs_allowed_median,away_runs_allowed_total,away_L,away_T,away_W,away_win_pct,away_run_diff_total,away_run_diff_mean
0,Ada Forest Hills Eastern,7,6.428571,6.0,45.0,7,1.0,0.0,7.0,0.0,...,13,1.769231,1.0,23.0,1.0,6.0,12.0,0.923077,113.0,8.692308
1,Addison,1,4.000000,4.0,4.0,1,5.0,5.0,5.0,1.0,...,15,4.533333,3.0,68.0,6.0,1.0,8.0,0.533333,26.0,1.733333
2,Adrian,1,0.000000,0.0,0.0,1,1.0,1.0,1.0,1.0,...,9,2.555556,2.0,23.0,2.0,9.0,7.0,0.777778,48.0,5.333333
3,Adrian Lenawee Christian,1,0.000000,0.0,0.0,1,7.0,7.0,7.0,1.0,...,5,13.200000,15.0,66.0,4.0,0.0,1.0,0.200000,-30.0,-6.000000
4,Adrian Madison,2,2.500000,2.5,5.0,2,2.5,2.5,5.0,1.0,...,8,5.625000,4.5,45.0,4.0,6.0,4.0,0.500000,5.0,0.625000


In [ ]:
# print a summary of how many teams are represented in each df len(df['teamName'].unique())

print('Number of teams in the original dataframe:', len(df['teamName'].unique()))
print('Number of teams in the processed dataframe:', len(test_df['teamName'].unique()))



Number of teams in the original dataframe: 641
Number of teams in the processed dataframe: 594


In [ ]:
### SAVE A TEMP DATAFRAME TO CSV
file_out = base_dir + '_summary_' + str(year) + '.csv'

test_df.to_csv(file_out, index=False)




In [ ]:
# ## Block of all the stat processing code to make into a function
# def process_stats(df):
#     ## Create a column WIth the result of each game win, loss or tie

#     def calc_result(row):
#         if row['teamScore'] > row['opponentScore']:
#             return 'W'
#         elif row['teamScore'] < row['opponentScore']:
#             return 'L'
#         else:
#             return 'T'

#     df['result'] = df.apply(calc_result, axis=1)

#     ## New Stragety - split the dataframe into playoff games, regular season games

#     ## playoff dataframe
#     playoff_df = df[df['postSeasonInfo'].notnull()]


#     playoff_df['homeOrAway'] = 'N'


#     ## regular season dataframe
#     regular_season_df = df[df['postSeasonInfo'].isnull()]

#     # Create home and away dataframes for the regular season dataframes
#     home_df = regular_season_df[regular_season_df['homeOrAway'] == 'H']   # home games
#     away_df = regular_season_df[regular_season_df['homeOrAway'] == 'A']   # away games


#     # Create a new dataframe with the summary stats for each team and each dataframe
#     ####### PLAYOFFS #######
#     grouped = playoff_df.groupby('teamName')

#     ## Save value counts to a new dataframe with the team name as the index and columns for each result type
#     results_frame = pd.DataFrame(grouped['result'].value_counts())

#     # transpose results into seperate columns for each result type
#     results_frame = results_frame.unstack(level=-1)

#     playoff_stat = grouped.agg({
#         'teamScore': ['count', 'mean', 'median', 'sum'],
#         'opponentScore': ['count', 'mean', 'median', 'sum']
#     }).reset_index()




#     # playoff_stat = df[['teamName', 'teamId', 'leagueName']].drop_duplicates()

#     # simplify the column names
#     playoff_stat.columns = ['teamName', 
                            
#                             'playoff_games_played',
#                             'playoff_runs_scored_mean',
#                             'playoff_runs_scored_median',
#                             'playoff_runs_scored_total',

#                             'playoff_runs_allowed_count',
#                             'playoff_runs_allowed_mean',
#                             'playoff_runs_allowed_median',
#                             'playoff_runs_allowed_total']


#     ## Add the result count columns to the playoff_stat dataframe
#     playoff_stat = playoff_stat.merge(results_frame, how='left', on='teamName')

#     ## Rename the columns
#     playoff_stat.columns = ['teamName',
                            
#                             'playoff_games_played',
#                             'playoff_runs_scored_mean',
#                             'playoff_runs_scored_median',
#                             'playoff_runs_scored_total',

#                             'playoff_runs_allowed_count',
#                             'playoff_runs_allowed_mean',
#                             'playoff_runs_allowed_median',
#                             'playoff_runs_allowed_total',
                            
#                             'playoff_losses',
#                             'playoff_wins'
#                             ]

#     # Calculate winning percentage and run differential (total and mean)
#     playoff_stat['playoff_win_pct'] = playoff_stat['playoff_wins'] / playoff_stat['playoff_games_played']
#     playoff_stat['playoff_run_diff_total'] = playoff_stat['playoff_runs_scored_total'] - playoff_stat['playoff_runs_allowed_total']
#     playoff_stat['playoff_run_diff_mean'] = playoff_stat['playoff_runs_scored_mean'] - playoff_stat['playoff_runs_allowed_mean']

#     ## Reorder the columns so the result columns right after the games played column
#     playoff_stat = playoff_stat[['teamName',
#                             'playoff_games_played',
#                             'playoff_wins',
#                             'playoff_losses',
#                             'playoff_win_pct',
#                             'playoff_runs_scored_mean', 
#                             'playoff_runs_scored_median',
#                             'playoff_runs_scored_total',
#                             'playoff_runs_allowed_mean',
#                             'playoff_runs_allowed_median',
#                             'playoff_runs_allowed_total',
#                             'playoff_run_diff_total',
#                             'playoff_run_diff_mean'
#                             ]]


#     ### Regular Season Stats#####

#     ## Create a new dataframe with the summary stats for each team and each dataframe
#     grouped = regular_season_df.groupby('teamName')

#     # Save the total games played for each team
#     regular_season_games_played = grouped['teamName'].count()

#     ## Save value counts to a new dataframe with the team name as the index and columns for each result type
#     results_frame = pd.DataFrame(grouped['result'].value_counts())

#     # # transpose results into seperate columns for each result type W L T
#     results_frame = results_frame.unstack(level=-1)

#     # add the regular_season_games_played
#     results_frame['regular_season_games_played'] = regular_season_games_played

#     # Rename reults columns
#     results_frame.columns = ['regular_season_losses',
#                             'regular_season_ties',
#                             'regular_season_wins',
#                             'regular_season_games_played']


#     ## Save the summary stats to a new dataframe
#     regular_season_stat = grouped.agg({
#         'teamScore': ['mean', 'median', 'sum'],
#         'opponentScore': ['mean', 'median', 'sum']
#     }).reset_index()



#     # simplify the column names
#     regular_season_stat.columns = ['teamName',
                            
#                             'regular_season_runs_scored_mean',
#                             'regular_season_runs_scored_median',
#                             'regular_season_runs_scored_total',
                            
#                             'regular_season_runs_allowed_mean',
#                             'regular_season_runs_allowed_median',
#                             'regular_season_runs_allowed_total']

#     ## Add the result count columns to the regular_season_stat dataframe
#     regular_season_stat = regular_season_stat.merge(results_frame, how='left', on='teamName')


#     # Calculate winning percentage and run differential (total and mean)
#     regular_season_stat['regular_season_win_pct'] = regular_season_stat['regular_season_wins'] / regular_season_stat['regular_season_games_played']
#     regular_season_stat['regular_season_run_diff_total'] = regular_season_stat['regular_season_runs_scored_total'] - regular_season_stat['regular_season_runs_allowed_total']
#     regular_season_stat['regular_season_run_diff_mean'] = regular_season_stat['regular_season_runs_scored_mean'] - regular_season_stat['regular_season_runs_allowed_mean']

#     ## Reorder the columns so the result columns right after the games played column
#     regular_season_stat = regular_season_stat[['teamName',
#                             'regular_season_games_played',
#                             'regular_season_wins',
#                             'regular_season_losses',
#                             'regular_season_ties',
#                             'regular_season_win_pct',
#                             'regular_season_runs_scored_mean',
#                             'regular_season_runs_scored_median',
#                             'regular_season_runs_scored_total',
#                             'regular_season_runs_allowed_mean',
#                             'regular_season_runs_allowed_median',
#                             'regular_season_runs_allowed_total',
#                             'regular_season_run_diff_total',
#                             'regular_season_run_diff_mean'
#                             ]]


#     ### Combine the playoff and regular season stats into one dataframe

#     ## Merge the playoff and regular season stats into one dataframe
#     combined_stat = playoff_stat.merge(regular_season_stat, how='left', on='teamName')

#     combined_stat.head()

#     ## Create a new dataframe with the summary stats for home and away games
#     # home dataframe

#     grouped = home_df.groupby('teamName')

#     # head
#     grouped.head()

#     ## Save the summary stats to a new dataframe
#     home_stat = grouped.agg({
#         'teamScore': ['mean', 'median', 'sum'],
#         'opponentScore': ['mean', 'median', 'sum']
#     }).reset_index()

#     # simplify the column names
#     home_stat.columns = ['teamName',
#                         'home_runs_scored_mean',
#                         'home_runs_scored_median',
#                         'home_runs_scored_total',

#                         'home_runs_allowed_mean',
#                         'home_runs_allowed_median',
#                         'home_runs_allowed_total']

#     ## Add the result count columns to the home_stat dataframe
#     home_stat = home_stat.merge(results_frame, how='left', on='teamName')


#     # Save the total games played for each team
#     home_games_played = grouped['teamName'].count()

#     ## Save value counts to a new dataframe with the team name as the index and columns for each result type
#     results_frame = pd.DataFrame(grouped['result'].value_counts())

#     # # # transpose results into seperate columns for each result type W L T
#     results_frame = results_frame.unstack(level=-1)

#     # simplify the column names
#     results_frame.columns = ['home_losses',
#                             'home_ties',
#                             'home_wins']

#     # # add the home_games_played
#     results_frame['home_games_played'] = home_games_played

#     # # Reorder reults columns
#     results_frame = results_frame[['home_games_played',
#                                 'home_wins',
#                                 'home_losses',
#                                 'home_ties']]





#     # Calculate winning percentage
#     results_frame['home_win_pct'] = results_frame['home_wins'] / results_frame['home_games_played']

#     ## add the results columns to the home_stat dataframe
#     home_stat = home_stat.merge(results_frame, how='left', on='teamName')

#     # Calculate run differential (total and mean)
#     home_stat['home_run_diff_total'] = home_stat['home_runs_scored_total'] - home_stat['home_runs_allowed_total']
#     home_stat['home_run_diff_mean'] = home_stat['home_runs_scored_mean'] - home_stat['home_runs_allowed_mean']



#     # drop the rehular season games played column
#     # home_stat.drop(columns=['regular_season_losses','regular_season_ties','regular_season_wins','regular_season_games_played'], inplace=True)


#     # reorder the columns
#     home_stat = home_stat[['teamName',
#                             'home_games_played',
#                             'home_wins',
#                             'home_losses',
#                             'home_ties',
#                             'home_win_pct',

#                             'home_runs_scored_mean',
#                             'home_runs_scored_median',
#                             'home_runs_scored_total',

#                             'home_runs_allowed_mean',
#                             'home_runs_allowed_median',
#                             'home_runs_allowed_total',

#                             'home_run_diff_total',
#                             'home_run_diff_mean'
#                             ]]


#     ### Merge the home stats into the combined dataframe
#     combined_stat = combined_stat.merge(home_stat, how='left', on='teamName')


#     ### Calculate the away stats

#     ## Create a new dataframe with the summary stats for away games
#     # away dataframe

#     grouped = away_df.groupby('teamName')

#     # head
#     grouped.head()


#     ## Save the summary stats to a new dataframe
#     away_stat = grouped.agg({
#         'teamScore': ['mean', 'median', 'sum'],
#         'opponentScore': ['mean', 'median', 'sum']
#     }).reset_index()

#     # simplify the column names
#     away_stat.columns = ['teamName',
#                         'away_runs_scored_mean',
#                         'away_runs_scored_median',
#                         'away_runs_scored_total',

#                         'away_runs_allowed_mean',
#                         'away_runs_allowed_median',
#                         'away_runs_allowed_total']

#     ## Add the result count columns to the home_stat dataframe
#     away_stat = away_stat.merge(results_frame, how='left', on='teamName')


#     # Save the total games played for each team
#     away_games_played = grouped['teamName'].count()

#     ## Save value counts to a new dataframe with the team name as the index and columns for each result type
#     results_frame = pd.DataFrame(grouped['result'].value_counts())

#     # # # transpose results into seperate columns for each result type W L T
#     results_frame = results_frame.unstack(level=-1)

#     # simplify the column names
#     results_frame.columns = ['away_losses',
#                             'away_ties',
#                             'away_wins']

#     # # add the home_games_played
#     results_frame['away_games_played'] = away_games_played

#     # # Reorder reults columns
#     results_frame = results_frame[['away_games_played',
#                                 'away_wins',
#                                 'away_losses',
#                                 'away_ties']]





#     # Calculate winning percentage
#     results_frame['away_win_pct'] = results_frame['away_wins'] / results_frame['away_games_played']

#     ## add the results columns to the home_stat dataframe
#     away_stat = away_stat.merge(results_frame, how='left', on='teamName')

#     # Calculate run differential (total and mean)
#     away_stat['away_run_diff_total'] = away_stat['away_runs_scored_total'] - away_stat['away_runs_allowed_total']
#     away_stat['away_run_diff_mean'] = away_stat['away_runs_scored_mean'] - away_stat['away_runs_allowed_mean']



#     # drop the rehular season games played column
#     # away_stat.drop(columns=['regular_season_losses','regular_season_ties','regular_season_wins','regular_season_games_played'], inplace=True)


#     # reorder the columns
#     away_stat = away_stat[['teamName',
#                             'away_games_played',
#                             'away_wins',
#                             'away_losses',
#                             'away_ties',
#                             'away_win_pct',

#                             'away_runs_scored_mean',
#                             'away_runs_scored_median',
#                             'away_runs_scored_total',

#                             'away_runs_allowed_mean',
#                             'away_runs_allowed_median',
#                             'away_runs_allowed_total',

#                             'away_run_diff_total',
#                             'away_run_diff_mean'
#                             ]]


#     ## Merge the away stats into the combined dataframe
#     combined_stat = combined_stat.merge(away_stat, how='left', on='teamName')

#     ## End the function
#     return combined_stat






